# Chaotic Attractors — Pairwise Distance Quality

Bifiltration: $\sqrt{\alpha}$ + eccentricity (Alpha complex).
$n = 1000$, noise $= 0$, 200 samples per class (1000 total).
CMD, MD 121, MD 10 for all pairs, separately for $H_0$ and $H_1$.

In [5]:
import numpy as np
import gudhi as gd
from scipy.integrate import solve_ivp
from sklearn.metrics import pairwise_distances as pdist_sklearn
from scipy.stats import spearmanr
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

In [6]:
n = 500
n_ex = 200
ALPHA_CLIP = 0.15

t_values = np.linspace(0, 1, 11)
a_fine = np.linspace(0.05, 0.95, 11)
b_fine = np.linspace(0, 1, 11)
a_coarse = np.array([0.25, 0.75])
b_coarse = np.linspace(0, 1, 5)
BOTTLENECK_E = 0.01

n_cmd = len(t_values)
n_md_fine = len(a_fine) * len(b_fine)
n_md_coarse = len(a_coarse) * len(b_coarse)
print(f'CMD: {n_cmd} | MD fine: {n_md_fine} | MD coarse: {n_md_coarse}')

CMD: 11 | MD fine: 121 | MD coarse: 10


In [7]:
def lorenz_rhs(t, s):
    x, y, z = s
    return [10.0*(y-x), x*(28.0-z)-y, x*y-8.0/3.0*z]
def rossler_rhs(t, s):
    x, y, z = s
    return [-(y+z), x+0.2*y, 0.2+z*(x-5.7)]
def thomas_rhs(t, s):
    x, y, z = s
    b = 0.208186
    return [np.sin(y)-b*x, np.sin(z)-b*y, np.sin(x)-b*z]
def sprott_rhs(t, s):
    x, y, z = s
    return [y+2.07*x*y+x*z, 1.0-1.79*x**2+y*z, x-x**2-y**2]
def fourwing_rhs(t, s):
    x, y, z = s
    return [0.2*x+y*z, 0.01*x-0.4*y-x*z, -z-x*y]

attractor_configs = [
    (lorenz_rhs,   20, 100, 5.0),
    (rossler_rhs,  50, 200, 1.0),
    (thomas_rhs,  100, 500, 1.0),
    (sprott_rhs,   50, 200, 0.5),
    (fourwing_rhs, 50, 300, 0.1),
]
attractor_names = ['Lorenz', 'Rössler', 'Thomas', 'Sprott', 'Four-Wing']
n_classes = len(attractor_configs)

def generate_attractor(cls_idx, n_pts=1000, seed=None):
    rhs, t_burn, t_run, x0_scale = attractor_configs[cls_idx]
    rng = np.random.RandomState(seed)
    x0 = rng.uniform(-x0_scale, x0_scale, 3)
    sol0 = solve_ivp(rhs, [0, t_burn], x0, rtol=1e-8, atol=1e-8, dense_output=True)
    y0 = sol0.y[:, -1]
    n_dense = max(10000, 20*n_pts)
    t_eval = np.linspace(0, t_run, n_dense)
    sol = solve_ivp(rhs, [0, t_run], y0, t_eval=t_eval, rtol=1e-8, atol=1e-8)
    traj = sol.y.T
    idx = np.linspace(0, len(traj)-1, n_pts, dtype=int)
    pts = traj[idx].copy()
    pmin, pmax = pts.min(axis=0), pts.max(axis=0)
    rng_vals = pmax - pmin
    rng_vals[rng_vals < 1e-12] = 1.0
    pts = (pts - pmin) / rng_vals
    return pts

In [8]:
Data, Labels = [], []
for cls_idx in range(n_classes):
    for s in range(n_ex):
        Labels.append(cls_idx)
        Data.append(generate_attractor(cls_idx, n_pts=n, seed=s*1000+cls_idx))
Labels = np.array(Labels)
N = len(Data)
print(f'N = {N} ({n_classes} classes x {n_ex} samples)')

N = 1000 (5 classes x 200 samples)


In [9]:
class CachedAlpha:
    def __init__(self, points):
        ac = gd.AlphaComplex(points=points)
        self.st = ac.create_simplex_tree()
        self._simplices = []
        self._alpha_vals = []
        for s, fv in self.st.get_simplices():
            self._simplices.append(tuple(s))
            self._alpha_vals.append(self.st.filtration(s))
        self._alpha_vals = np.array(self._alpha_vals)
        max_dim = max(len(s) for s in self._simplices)
        self._vert_idx = np.full((len(self._simplices), max_dim), -1, dtype=np.int32)
        for i, s in enumerate(self._simplices):
            self._vert_idx[i, :len(s)] = s
    def compute_pd(self, func_values):
        f_ext = np.append(func_values, -np.inf)
        filt_vals = np.max(f_ext[self._vert_idx], axis=1)
        for i, s in enumerate(self._simplices):
            self.st.assign_filtration(s, float(filt_vals[i]))
        self.st.make_filtration_non_decreasing()
        self.st.persistence()
        pds = []
        for dim in range(2):
            pd = self.st.persistence_intervals_in_dimension(dim)
            if pd is not None and len(pd) > 0:
                pd = pd[np.isfinite(pd[:, 1])]
            if pd is None or len(pd) == 0:
                pd = np.empty((0, 2))
            pds.append(pd)
        return tuple(pds)

def phi_star_ab(phi1, phi2, a, b):
    return np.minimum(a, 1-a) * np.maximum((phi1-b)/a, (phi2+b)/(1-a))

def safe_bottleneck(pd1, pd2, e=0.01):
    if len(pd1) == 0 and len(pd2) == 0: return 0.0
    if len(pd1) == 0: return float(np.max((pd2[:,1]-pd2[:,0])/2))
    if len(pd2) == 0: return float(np.max((pd1[:,1]-pd1[:,0])/2))
    return gd.bottleneck_distance(pd1, pd2, e)

In [10]:
# Filtrations: sqrt(alpha) clipped + eccentricity
Phi1, Phi2 = [], []
Alphas = []
for data in tqdm(Data, desc='Filtrations'):
    ac = gd.AlphaComplex(points=data)
    st = ac.create_simplex_tree()
    alpha_vals = np.array([st.filtration([v]) for v in range(len(data))])
    alpha_sqrt = np.sqrt(np.clip(alpha_vals, 0, None))
    alpha_sqrt = np.clip(alpha_sqrt, 0, ALPHA_CLIP)
    alpha_sqrt = alpha_sqrt / ALPHA_CLIP  # normalize to [0,1]
    Phi1.append(alpha_sqrt)
    ecc = np.max(pdist_sklearn(data), axis=1)
    ecc = (ecc - ecc.min()) / (ecc.max() - ecc.min()) if ecc.max() > ecc.min() else ecc
    Phi2.append(ecc)

def make_cmd_filts(p1, p2): return [(1-t)*p1 + t*p2 for t in t_values]
def make_mdf_filts(p1, p2): return [phi_star_ab(p1, p2, a, b) for a in a_fine for b in b_fine]
def make_mdc_filts(p1, p2): return [phi_star_ab(p1, p2, a, b) for a in a_coarse for b in b_coarse]

params = [('cmd', make_cmd_filts, n_cmd),
          ('mdf', make_mdf_filts, n_md_fine),
          ('mdc', make_mdc_filts, n_md_coarse)]

PDs = {name: [None]*N for name, _, _ in params}
for i in tqdm(range(N), desc='PDs'):
    cst = CachedAlpha(Data[i])
    phi1, phi2 = Phi1[i], Phi2[i]
    for name, make_filts, _ in params:
        PDs[name][i] = [cst.compute_pd(f) for f in make_filts(phi1, phi2)]

PDs: 100%|██████████| 1000/1000 [35:53<00:00,  2.15s/it] 


In [11]:
Ds = {}
for name, _, nparam in params:
    print(f'Distances: {name}')
    D_H0 = np.zeros((N, N))
    D_H1 = np.zeros((N, N))
    for i in tqdm(range(N), leave=False):
        for j in range(i+1, N):
            max_d0 = max_d1 = 0.0
            for p in range(nparam):
                d0 = safe_bottleneck(PDs[name][i][p][0], PDs[name][j][p][0], BOTTLENECK_E)
                d1 = safe_bottleneck(PDs[name][i][p][1], PDs[name][j][p][1], BOTTLENECK_E)
                if d0 > max_d0: max_d0 = d0
                if d1 > max_d1: max_d1 = d1
            D_H0[i,j] = D_H0[j,i] = max_d0
            D_H1[i,j] = D_H1[j,i] = max_d1
    Ds[name] = {'H0': D_H0, 'H1': D_H1}

Distances: cmd


Distances: mdf


Distances: mdc


In [12]:
np.save('Attractors_distances.npy', {
    'd_cmd_H0': Ds['cmd']['H0'], 'd_cmd_H1': Ds['cmd']['H1'],
    'd_mdf_H0': Ds['mdf']['H0'], 'd_mdf_H1': Ds['mdf']['H1'],
    'd_mdc_H0': Ds['mdc']['H0'], 'd_mdc_H1': Ds['mdc']['H1'],
    'labels': Labels
})

In [13]:
triu = np.triu_indices(N, k=1)
for hdim in ['H0', 'H1']:
    d_cmd = Ds['cmd'][hdim][triu]
    d_mdf = Ds['mdf'][hdim][triu]
    d_mdc = Ds['mdc'][hdim][triu]
    sp_cmd_mdf, _ = spearmanr(d_mdf, d_cmd)
    sp_mdc_mdf, _ = spearmanr(d_mdf, d_mdc)
    sp_cmd_mdc, _ = spearmanr(d_cmd, d_mdc)
    print(f'\n{hdim}:')
    print(f'  Spearman CMD vs MD121: {sp_cmd_mdf:.4f}')
    print(f'  Spearman MD10 vs MD121: {sp_mdc_mdf:.4f}')
    print(f'  Spearman CMD vs MD10: {sp_cmd_mdc:.4f}')


H0:
  Spearman CMD vs MD121: 0.9999
  Spearman MD10 vs MD121: 0.9998
  Spearman CMD vs MD10: 0.9997

H1:
  Spearman CMD vs MD121: 0.9958
  Spearman MD10 vs MD121: 0.9915
  Spearman CMD vs MD10: 0.9819
